# 01 - Sentiment Analysis with Python

### Setting up credentials to use Google Cloud

In [ ]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "your-credentials.json"

assert os.getenv("GOOGLE_APPLICATION_CREDENTIALS") != "your-credentials.json", "Please set your credentials path"

### Running query to BigQuery from Jupyter notebook

In [ ]:
# load the Google Cloud Bigquery extension for jupyter notebook
%load_ext google.cloud.bigquery

try querying 10 rows to test the connection


In [ ]:
%%bigquery

SELECT
    review,
    label
FROM
    `bigquery-public-data.imdb.reviews`
WHERE
    label IN ("Negative", "Positive")
    AND split = "train"
LIMIT 10

### Querying train dataset

In [ ]:
%%bigquery train_df

SELECT
    DISTINCT review,
    label
FROM
    `bigquery-public-data.imdb.reviews`
WHERE
    label IN ("Negative", "Positive")
    AND split = "train"

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df['label'].value_counts()

### Querying test dataset

In [ ]:
%%bigquery test_df

SELECT
    DISTINCT review,
    label
FROM
    `bigquery-public-data.imdb.reviews`
WHERE
    label IN ("Negative", "Positive")
    AND split = "test"

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df.info()

In [ ]:
test_df.describe()

In [ ]:
test_df['label'].value_counts()

### Training a classification model

In [ ]:
import pandas as pd

from sklearn import linear_model, metrics
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# create count vector with regex token pattern of any word with more than 2 a-z characters
count_vectorizer = CountVectorizer(token_pattern="[a-z]{2,}")

# prepare train dataset (features and labels)
x_train = count_vectorizer.fit_transform(train_df['review'])
y_train = train_df['label'].apply(lambda x: 1 if x == "Negative" else 0)

# prepare test dataset (features and labels)
x_test = count_vectorizer.transform(test_df['review'])
y_test = test_df['label'].apply(lambda x: 1 if x == "Negative" else 0)

In [ ]:
# visualize bag-of-words features produced from sklearn
df_bow_sklearn = pd.DataFrame(x_train.toarray(), columns=count_vectorizer.get_feature_names_out())
df_bow_sklearn.head()

In [ ]:
# train a logistic regression model using train dataset
model = linear_model.LogisticRegression(solver="lbfgs", max_iter=1000)
model.fit(x_train, y_train)

In [ ]:
# test the trained model using test dataset
predictions = model.predict(x_test)

accuracy = metrics.accuracy_score(predictions, y_test)
f1_score = metrics.f1_score(predictions, y_test, average='macro')

accuracy, f1_score